In [3]:
#STEP 1 CREATE DATA FOR TRAINING THE MODEL
#Create Training Data
import numpy as np
import cv2
import time

# load haarcascade face classifier
face_classifier = cv2.CascadeClassifier( "haarcascade_frontalface_default.xml" )
 
# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is () :
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        
    return cropped_face




# Collect 100 samples of your face from webcam input
def collectingFaces(path, count):
    # Initialize Webcam
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if face_extractor(frame) is not None:
            count += 1
            face = cv2.resize(face_extractor(frame), (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
            # Save file in specified directory with unique name
            file_name_path = path + str(count) + '.jpg'
            cv2.imwrite(file_name_path, face)
        
            # Put count on images and display live count
            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Cropper', face)
        
        else:
            print("Face not found")
            pass
        if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
            break
        
    cap.release()
    cv2.destroyAllWindows()      
    print("Collecting Samples Complete")


i = 0 
j = 0

print("Collecting Samples Of Rohit")
collectingFaces("./faces/rohit/", i)
print("Wait for 10 Second....")
time.sleep(10)

print("Collecting Samples Of Rahul")
collectingFaces("./faces/rahul/", j)

<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-981f40a91afe>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is () :


Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Wait for 10 Second....


In [4]:
# STEP 2 TRAIN MODEL FROM THE CREATED TRAINING DATA
#Train Model
from sys import path
import cv2
import numpy as np
from os import listdir
from os.path  import isfile, join
# Get training data we previously made
def trainModel(path):
    
    data_path = path
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    # Create arrays from training data and labels
    Training_Data, Labels = [], []

    # Open training images in our datapath
    # Create a numpy array for training data
    for i, files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)
    
    # Create a numpy array for both training data and labels
    Labels = np.asarray(Labels, dtype=np.int32)

    # Initialize facial recognizer
    # model = cv2.face.createLBPHFaceRecognizer()
    # NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
    # pip install opencv-contrib-python
    # model = cv2.createLBPHFaceRecognizer()
    
    model = cv2.face_LBPHFaceRecognizer.create()
    # Let's train our model 
    model.train(np.asarray(Training_Data), np.asarray(Labels))
    return model
  


rohit_model = trainModel("./faces/rohit/")
print("Rohit's Model trained sucessefully")
rahul_model = trainModel("./faces/rahul/")
print("Rahul's Model trained sucessefully")

Rohit's Model trained sucessefully
Rahul's Model trained sucessefully


In [ ]:
import cv2
import numpy as np
import os
import pywhatkit

def confidence(results, image):
    if results[1] < 500 :
        confidence = int( 100 * (1 - (results[1])/400) )
        display_string = str(confidence) + '% Confident it is User'
        cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
    return confidence

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is () :
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


def sendingmail(to, text):
    import smtplib  
    server = smtplib.SMTP_SSL( "smtp.gmail.com", 465 )
    server.login( "1824004062.rohit@gmail.com", "1824004062" )
    server.sendmail( "1824004009rahulkashyap@gmail.com", to, text)
    server.quit()

def get_time():
    import time
    t = time.localtime()
    hour = int(time.strftime("%H", t))
    min = int(time.strftime("%M", t)) +1 
    return hour, min

# Open Webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        rohit_results = rohit_model.predict(face)
        rahul_results = rahul_model.predict(face)
        
        rohit_confidence = confidence(rohit_results, image)
        rahul_confidence = confidence(rahul_results, image)

        if  rohit_confidence > 50:
            cv2.putText(image, "Rohit Vishwakarma ", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            cv2.waitKey(3000)
            cv2.destroyAllWindows()
            sendingmail("1824004062.rohit@gmail.com", "Hey cage!!,  your face detected in camera!")
            os.system("aws ec2 run-instances --image-id ami-0ad704c126371a549 --instance-type t2.micro")
            os.system("aws ec2 create-volume --volume-type gp2 --size 5 --region ap-south-1 --availability-zone ap-south-1a")
            i = input("Enter VolumeID: ")
            j = input("Enter InstanceID: ")
            command = "aws ec2 attach-volume --volume-id " + i + " --instance-id " + j + " --device /dev/sdf --region ap-south-1"
            os.system(command)
            break
         
        elif rahul_confidence > 40:
            cv2.putText(image, "Rahul Kashyap", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            cv2.waitKey(3000)
            cv2.destroyAllWindows()
            sendingmail("1824004062.rohit@gmail.com", "Hey cage!!,  your face detected in camera!")
            pywhatkit.sendwhatmsg_instantly("+919033946703", "Helloo!!, This is from Model?")


        else:    
            cv2.putText(image, "Rcognizing Face...", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )
        
    except:
        cv2.putText(image, "No Face Found!", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "Looking For Face...", (220, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows() 

<>:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-6-c75f1cbf7468>:20: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is () :
